# Install

In [ ]:
!pip3 install soynlp

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [1]:
import sys
sys.path.append('drive/MyDrive/MemeProject/src/doc2vec/')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', 'drive/MyDrive/MemeProject/src/doc2vec/']


In [11]:
import re
import config as cfg
import swifter
from pshmodule.utils import filemanager as fm

In [3]:
df = fm.load(cfg.temp_data_colab)

extension : .pickle
Loaded 10317 records from drive/MyDrive/MemeProject/data/temp_for_doc2vec.pickle


In [4]:
df.head()

,content,okt,soynlp
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반, 1등]","[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]","[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]","[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]","[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]","[중간고사, 반, 1등, 먹음]"


In [5]:
df_ref = fm.load(cfg.temp_ref_data_colab)

extension : .pickle
Loaded 208700 records from drive/MyDrive/MemeProject/data/temp_for_ref.pickle


In [6]:
df_ref.cpr.fillna('', inplace=True)
df_ref.cpo.fillna('', inplace=True)

In [13]:
df_ref.head()

,u,cpr,mm,cpo
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,


In [12]:
df_ref['u'] = df_ref.u.swifter.apply(lambda x: re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', x))

Pandas Apply:   0%|          | 0/208700 [00:00<?, ?it/s]

# Model Load

In [14]:
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec


In [15]:
# jvm_path = "/Library/Java/JavaVirtualMachines/zulu-15.jdk/Contents/Home/bin/java"
# okt = Okt(jvmpath=jvm_path)
okt = Okt()

In [16]:
model_okt = Doc2Vec.load(cfg.doc2vec_okt_colab)

In [17]:
model_soynlp = Doc2Vec.load(cfg.doc2vec_soynlp_colab)

# Memebot with doc2vec Similarity

### 학습된 문장 유사도

In [26]:
input_sent = input()

오늘 날씨가 너무 좋다


##### okt

In [27]:
index = 4
print(df.content.iloc[index])
print("-"*100)
result = model_okt.docvecs.most_similar(index)
for k, v in enumerate(result):
    if k == 0:
        print(f"k : {k}")
        print(f"v : {v}")
        print(df.content.iloc[int(v[0])])

중간고사 반 1등 먹음
----------------------------------------------------------------------------------------------------
k : 0
v : ('0', 0.915203332901001)
나 중간고사 반에서 1등했어


In [28]:
df_ref.tail()

,u,cpr,mm,cpo
208695,전국노래자랑에서 이모가 1등했다,대길이 이모님... 힘숨찐ㄷㄷ,상상도 못한 정체 ㄴㅇㄱ,
208696,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시네!,상상도 못한 정체 ㄴㅇㄱ,
208697,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시라능,상상도 못한 정체 ㄴㅇㄱ,
208698,전국노래자랑에서 이모가 1등했다,대길좌 이모님이 우승자라구?!,상상도 못한 정체 ㄴㅇㄱ,
208699,전국노래자랑에서 이모가 1등했다,"대길쓰 이모님,, 힘숨찐ㄷㄷ",상상도 못한 정체 ㄴㅇㄱ,


입력 문장 유사도

In [31]:
# vectorize new sentences based on doc2vec
input_vec = model_okt.infer_vector(okt.morphs(input_sent))

# most similarity top 5
return_docs = model_okt.docvecs.most_similar(positive=[input_vec], topn=5)

# find Similar Sentences in the Trained Dataframe
result = [df.content.iloc[int(i[0])] for i in return_docs]

# result
if df_ref.empty:
    print("유사 문장이 없습니다.")
else:
    print(f"입력 문장 : {input_sent}")
    print("-"*100)
    for k, i in enumerate(zip(return_docs, result)):
        print(f"{k+1}.")
        print(f"유사 문장 : {i[1]}")
        print(f"유사 문장 : {i[0][1]}")
        # find Similar Sentences in Dialogueset
        temp_ref = df_ref[df_ref.u == i[1]]
        temp_ref = temp_ref.sample(frac=1).reset_index(drop=True)
        result = str(temp_ref.cpr.iloc[0]) + " " + str(temp_ref.mm.iloc[0]) + " " + str(temp_ref.cpo.iloc[0])
        print(f"밈 답변 : {result.strip()}")
        print("-"*100)

입력 문장 : 오늘 날씨가 너무 좋다
----------------------------------------------------------------------------------------------------
1.
유사 문장 : 오늘 날씨가 진짜 너무 좋은데
유사 문장 : 0.6323586702346802
밈 답변 : 내 덕분인 거 아니? 네가 말로만 듣던 걔가 나야↗ 날씨요정♬
----------------------------------------------------------------------------------------------------
2.
유사 문장 : 너 너무 능력이 없어
유사 문장 : 0.555325448513031
밈 답변 : 하, 왜 또 이래? 저기압일 땐 고기 앞으로~
----------------------------------------------------------------------------------------------------
3.
유사 문장 : 너 너무 못 알아듣네
유사 문장 : 0.5541388988494873
밈 답변 : 그래도 이만하면 괜찮쟈나~~ 내 장점이 뭔지 알아↗ 바로 뻔뻔한 거야↘♪♪
----------------------------------------------------------------------------------------------------
4.
유사 문장 : 야 말을 너무 못하잖아
유사 문장 : 0.5521834492683411
밈 답변 : 그렇게 됐다...
----------------------------------------------------------------------------------------------------
5.
유사 문장 : 너 너무 재미가 없어
유사 문장 : 0.548125147819519
밈 답변 : 재미가 없으면... 그냥 꼬옥 안아주면 되...
---------------------------------------------

##### soynlp

In [32]:
index = 0
print(df.content.iloc[index])
print("-"*100)
result = model_soynlp.docvecs.most_similar(index)
for k, v in enumerate(result):
    if k == 0:
        print(f"k : {k}")
        print(f"v : {v}")
        print(df.content.iloc[int(v[0])])

나 중간고사 반에서 1등했어
----------------------------------------------------------------------------------------------------
k : 0
v : ('3550', 0.9001673460006714)
나 반에서 인기 투표 1위했어


입력 문장 유사도

In [33]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

ltokenizer setting

In [34]:
# WordExtractor
word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.load(cfg.soynlp)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

# 사용자 사전 추가
with open(cfg.user_dict, 'r') as f:
  user_dict = f.readlines()

user_dict = [i.replace('\n', '') for i in user_dict]

for i in user_dict:
  cohesion_score[i] = 1.0

# LTokenizer
tokenizer = LTokenizer(scores=cohesion_score)

all cohesion probabilities was computed. # words = 1856
all branching entropies was computed # words = 11743
all accessor variety was computed # words = 11743


In [35]:
# vectorize new sentences based on doc2vec
input_vec = model_soynlp.infer_vector(tokenizer.tokenize(input_sent))

# most similarity top 5
return_docs = model_soynlp.docvecs.most_similar(positive=[input_vec], topn=5)

# find Similar Sentences in the Trained Dataframe
result = [df.content.iloc[int(i[0])] for i in return_docs]

# result
if df_ref.empty:
    print("유사 문장이 없습니다.")
else:
    print(f"입력 문장 : {input_sent}")
    print("-"*100)
    for k, i in enumerate(zip(return_docs, result)):
        print(f"{k+1}.")
        print(f"유사 문장 : {i[1]}")
        print(f"유사 문장 : {i[0][1]}")
        # find Similar Sentences in Dialogueset
        temp_ref = df_ref[df_ref.u == i[1]]
        
        temp_ref = temp_ref.sample(frac=1).reset_index(drop=True)
        result = str(temp_ref.cpr.iloc[0]) + " " + str(temp_ref.mm.iloc[0]) + " " + str(temp_ref.cpo.iloc[0])
        print(f"밈 답변 : {result.strip()}")
        print("-"*100)

입력 문장 : 오늘 날씨가 너무 좋다
----------------------------------------------------------------------------------------------------
1.
유사 문장 : 앞으로 1주일 동안은 날씨가 괜찮다더라
유사 문장 : 0.5888290405273438
밈 답변 : 완전 맛도리 날씨 맛집! 한강에서 치맥할 파티원 모집 (1/N)
----------------------------------------------------------------------------------------------------
2.
유사 문장 : 오늘 날씨 좋다
유사 문장 : 0.5725225806236267
밈 답변 : 날씨 덕분에 기분도 좋아짐~! 소리 벗고 팬티 질러♬
----------------------------------------------------------------------------------------------------
3.
유사 문장 : 오늘 날씨가 너무 쾌청해
유사 문장 : 0.5583449602127075
밈 답변 : 네가 말로만 듣던 걔가 나야↗ 날씨요정♬
----------------------------------------------------------------------------------------------------
4.
유사 문장 : 날씨가 진짜 너무 좋은데
유사 문장 : 0.5495733022689819
밈 답변 : 크크 내 덕분이라는 거~~ 네가 말로만 듣던 걔가 나야↗ 날씨요정↗
----------------------------------------------------------------------------------------------------
5.
유사 문장 : 어저께 술 너무 달렸어
유사 문장 : 0.5194008350372314
밈 답변 : 발 귀에 대고 여보세요 한 건 기억남? 지가 이 두 눈으로 똑똑히 봤슈...!
------